In [1]:
import pandas as pd
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
X_all=np.load("drive/MyDrive/speaker/train_gray.npy")

X_all = X_all.astype('float16')
X_all = X_all/255

Y_all=pd.read_csv("drive/MyDrive/speaker/filelist.csv",index_col=0)

X_all.shape

Y_all.shape

test=np.loadtxt("drive/MyDrive/speaker/testing_list.txt",dtype="str")

test=np.char.replace(test,".wav","")

test=np.char.split(test,"/")

test_name=np.array([x[1] for x in test])
test_type=np.array([x[0] for x in test])

valid=np.loadtxt("drive/MyDrive/speaker/validation_list.txt",dtype="str")

valid=np.char.replace(valid,".wav","")

valid=np.char.split(valid,"/")

valid_name=np.array([x[1] for x in valid])
valid_type=np.array([x[0] for x in valid])

Y_all.Name=Y_all.Name.str.replace(".png","")

Y_all["Index"]=Y_all.index

valid=pd.DataFrame({"Name":valid_name,"Type":valid_type})

test=pd.DataFrame({"Name":test_name,"Type":test_type})

valid=pd.merge(Y_all,valid,on=["Name","Type"],how="right")

test=pd.merge(Y_all,test,on=["Name","Type"],how="right")

Y_all=Y_all[~Y_all.Index.isin(valid.Index)]
Y_all=Y_all[~Y_all.Index.isin(test.Index)]

x_train=X_all[Y_all.Index]

y_train=Y_all.Type.to_numpy()

classes=np.unique(y_train)

del X_all

del Y_all

In [4]:
commands=["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

In [5]:
y_train[np.isin(y_train,commands,invert=True)]="unknown"

In [6]:
from sklearn import preprocessing
pre = preprocessing.LabelEncoder()
pre.fit(y_train)
y_train = pre.transform(y_train)

In [7]:
subtract_pixel_mean = True

In [8]:
import tensorflow as tf
checkpoint_filepath = 'drive/MyDrive/speaker/checkpoint_model_resnet'

In [9]:
if subtract_pixel_mean:
    train_mean = np.mean(x_train, axis=0)

In [10]:
del x_train

In [11]:
x_test=np.load("drive/MyDrive/speaker/test_grey.npy")

In [12]:
x_test = x_test.astype('float16',copy=False)

In [13]:
x_test = x_test/255

In [14]:
if subtract_pixel_mean:
    x_test -= train_mean

In [15]:
model = tf.keras.models.load_model(checkpoint_filepath)

In [16]:
test_name=pd.read_csv("drive/MyDrive/speaker/filelist_test.csv",index_col=0)
test_name=test_name.Name.str.replace(".png",".wav").to_numpy()

In [17]:
for i in range(4,6):
  print(i)
  pred=np.argmax(model.predict(x_test[i::6],batch_size=1000),axis=1)
  pred=pre.inverse_transform(pred)
  results=pd.DataFrame({"fname":test_name[i::6],"label":pred})
  results.to_csv("drive/MyDrive/speaker/results"+str(i)+".csv",index=None)
  model.reset_states()

4
5


In [19]:
res0=pd.read_csv("drive/MyDrive/speaker/results0.csv")
res1=pd.read_csv("drive/MyDrive/speaker/results1.csv")
res2=pd.read_csv("drive/MyDrive/speaker/results2.csv")
res3=pd.read_csv("drive/MyDrive/speaker/results3.csv")
res4=pd.read_csv("drive/MyDrive/speaker/results4.csv")
res5=pd.read_csv("drive/MyDrive/speaker/results5.csv")

In [23]:
total_results=pd.concat([res0,res1,res2,res3,res4,res5])

In [24]:
total_results.to_csv("drive/MyDrive/speaker/results.csv",index=None)